(C:ssp-bruit)=
# Bruit dans les signaux

Jusqu'ici, on considérait les signaux comme étant déterministes, c'est à dire qu'on les connaissait de manière sûre.
Ils étaient définis par une forme analytique (par exemple : $x[n] = \sin(2\pi f n)$)
ou une suite d'échantillons (par exemple : $x[n] = [1, 2, 3, 4]$).

Mais en pratique, il est rare que les signaux soient connus exactement et parfaitement.
Les signaux aléatoires (c'est-à-dire non déterministes)
sont des signaux qu'on ne peut pas prédire complètement avant de les avoir observés.
Par exemple :
* un signal bruité (il contient des parasites) est un signal aléatoire,
  car le bruit n'est pas parfaitement connu
  (si c'était le cas, alors il serait très simple de supprimer le bruit du signal !) ;
* une sinusoïde de fréquence inconnue peut également être considérée comme un signal aléatoire ;
* un signal de communication numérique, au niveau du récepteur, est un signal aléatoire
  puisque le message émis n'est pas connu a priori du récepteur (sinon, il serait inutile de transmettre le message...).

Attention, lorsqu'on a mesuré et enregistré un signal physique, cet enregistrement n'est plus aléatoire.
Mais une deuxième mesure du même phénomène physique sera différente.
Lorsqu'on parle de signal aléatoire, c'est en fait le phénomène avant la mesure qui est aléatoire !
Ce qui nous intéresse ici, c'est de concevoir un traitement
sur un signal qui n'a pas encore été mesuré et qui est donc considéré aléatoire.

Pour analyser et traiter ces signaux, on se base sur leurs propriétés statistiques,
telles que leur moyenne ou une distribution de probabilité : nous allons donc commencer par quelques rappels de probabilités.
Enfin, nous ne considérons que les signaux discrets dans ce chapitre pour simplifier le propos.


<!--
[Jutten] DSP, stationnarité et ergodicité
Par définition, un signal aléatoire ne peut pas être décrit par une loi mathématique qui prédit sa valeur à chaque instant, car cette valeur n’est pas prédictible analytiquement. En revanche, on peut décrire ses propriétés à l’aide de probabilités et de statistiques. Mathématiquement, un signal aléatoire sera considéré comme la réalisation d’un processus aléatoire (random process), et la valeur prise à un instant ti comme une variable aléatoire (VA).
Un processus aléatoire (ou stochastique) est une famille de fonctions X(t), réelles ou complexes, définies dans un espace de probabilité, c’est-à-dire dépendant de deux variables, dont l’une est le temps t (usuellement, mais cela peut être l’espace pour des images), et dont l’autre est la variable ζ de l’espace des épreuves Ω.
(une VA est définie à partir de sa densité de probabilité) une variable aléatoire
X(ti) ou Xi, dont le comportement nécessite la connaissance de sa densité de probabilité. (espérance, moyenne et son estimation : Eq 6.7-6.9)
(Stationnarité au sens strict) Un processus aléatoire est dit stationnaire au sens strict si toutes ses propriétés statistiques sont invariantes à un changement d’origine du temps.
(Stationnarité au sens large) Un processus aléatoire est dit stationnaire au sens large si toutes ses propriétés statistiques d’ordre 1 et 2 sont invariantes à un changement d’origine du temps.
(Ergodicité) Un processus aléatoire est dit ergodique si les valeurs moyennes statistiques (d’ensemble sur ζ) sont égales aux valeurs moyennes temporelles (sur une seule réalisation). La conséquence de cette hypothèse est très importante en pratique. Elle permet de remplacer les calculs de moments statistiques (qui supposent connues les densités de probabilité ou les fonctions de répartition) par les moyennes temporelles sur une seule réalisation (observation). En pratique, faire une estimation de la moyenne temporelle sur une fenêtre de taille infinie est impossible. Il faut se contenter d’une approximation calculée sur une fenêtre de taille finie, qui tendra asymptotiquement, avec la taille de la fenêtre, vers la moyenne temporelle. Cette hypothèse d’ergodicité est cependant difficile à vérifier. On admettra fréquemment que les processus aléatoires usuels sont ergodiques.
(voir au début du chapitre la définition d'un processus aléatoire)
Comme pour les signaux certains à puissance moyenne finie, la transformée de Fourier d’une réalisation xi (t) d’un processus aléatoire n’existe pas, car l’intégrale ne converge pas. On peut définir la densité spectrale de puissance (DSP) comme la limite de la DSP sur une fenêtre de largeur T , lorsque T → +∞, si la limite existe
(Théorème de Wiener-Khintchine) La densité spectrale de puissance d’un processus aléatoire stationnaire au sens large est la transformée de Fourier de sa fonction d’autocorrélation.
Remarques
1. Ce théorème donne une définition unique de la DSP, valide pour des signaux certains et aléatoires.
2. Pour des signaux aléatoires, ce théorème fournit une méthode pratique de calcul de la DSP.
3. Si la DSP d’un signal aléatoire, Sxx(f), est connue, on peut en déduire la fonction d’autocorrélation Rxx(τ) par transformée de Fourier inverse
On appelle bruit blanc un processus aléatoire W dont la densité spectrale de puissance est constante, ∀f. Le bruit à DSP constante est dit blanc par analogie avec la lumière blanche qui contient toutes les longueurs d’onde de la lumière visible. Le bruit blanc correspond à un modèle purement théorique. En effet, il est physiquement irréalisable car il contient des fréquences infinies et sa puissance moyenne est infinie. C’est cependant un modèle très pratique et utile pour représenter des signaux dont le spectre est constant, au moins sur une large bande de fréquence.
(Théomème de la limite centrale) La distribution statistique d’une somme de n variables aléatoires indépendantes, possédant la même loi de probabilité, tend asymptotiquement (lorsque n → +∞) vers une distribution gaussienne, quelle que soit la distribution des variables individuelles. Ce théorème joue un rôle important en théorie du signal, car il justifie que de nombreux phénomènes aléatoires, résultant de la superposition de très nombreux phénomènes aléatoires individuels indépendants, soient modélisés par une loi gaussienne.
Exo possible : exo 6.7.4
-->

## Rappels de probabilité

Une _variable aléatoire_ est une variable mathématique dont la valeur n'est pas fixée.
Lors d'une expérience ou après l'acquisition d'un signal,
on dispose de mesures de la variable aléatoire sous forme de valeurs numériques :
c'est ce qu'on appelle des _réalisations_.

Les valeurs possibles d'une variable aléatoire n'ont pas forcément la même probabilité d'apparaître :
la distribution des probabilités d'apparition des valeurs est appelée _densité de probabilité_
et elle est souvent notée $p$.
Les densités de probabilité les plus courantes en traitement statistique du signal sont listées ci-dessous.


### Densité de probabilité gaussienne

La densité de probabilité gaussienne, ou loi normale, est la bien connue « courbe en cloche ».
En notant $x$ la variable aléatoire, elle a pour expression :

$$
p(x) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(x-\mu^2)}{2\sigma^2}\right),
$$

et dépend de deux paramètres : la moyenne $\mu$ et l'écart-type $\sigma$.
La moyenne représente la valeur moyenne que prend un échantillon, l'écart-type est l'écart moyen d'un échantillon par rapport à $\mu$.
Ainsi, la variable aléatoire peut prendre n'importe quelle valeur sur $\mathbb{R}$,
les valeurs proches de $\mu$ étant plus probables que les valeurs éloignées de $\mu$.

On note aussi :

$$
x \sim \mathcal{N}(\mu\ ;\sigma).
$$

```{figure} loi-normale.svg
---
name: F:ssp:normale
---
Densités de probabilité gaussiennes et 500 réalisations
(à gauche : $\mathcal{N}(0\ ;1)$, à droite : $\mathcal{N}(1\ ;0,3)$).
```

La densité de probabilité gaussienne est un très bon modèle pour la plupart des bruits courants.


### Densité de probabilité uniforme

Une variable aléatoire $x$ distribuée suivant la densité de probabilité uniforme
peut prendre n'importe quelle valeur sur un intervalle $[a,b]$, et aucune valeur de cet intervalle n'est privilégiée par rapport à une autre.
Elle a pour expression :

$$
p(x) = 
\begin{cases}
\frac{1}{b-a} &\text{si}\, x\in[a,b], \\
0             &\text{sinon}.
\end{cases}
$$

On note aussi :

$$
x \sim \mathcal{U}[a\ ;b].
$$


```{figure} loi-uniforme.svg
---
name: F:ssp:uniforme
---
Densités de probabilité uniformes et 500 réalisations
(à gauche : $\mathcal{U}[0\ ;1]$, à droite : $\mathcal{U}[-2\ ;2]$).
```

La distribution de probabilité uniforme a le mérite d'être très simple,
même si elle représente assez mal les bruits les plus classiques dans les signaux.
Si la variable aléatoire uniforme prend des valeurs discrètes,
sa distribution de probabilité peut modéliser des bruits de quantification.


### Loi de Poisson

Enfin, la loi de [Poisson](https://fr.wikipedia.org/wiki/Sim%C3%A9on_Denis_Poisson)
modélise un processus de comptage (par exemple un nombre de photons sur les pixels d'un capteur CCD).
Elle a pour expression :

$$
p(x) = \frac{\lambda^x}{x!} e^{-\lambda}
$$

où $\lambda$ est l'unique paramètre, il correspond à la fois à la moyenne et la variance (c'est-à-dire le carré de l'écart-type).

On note aussi :

$$
x \sim \mathcal{P}(\lambda).
$$

```{figure} loi-poisson.svg
---
name: F:ssp:poisson
---
Loi de Poisson et 500 réalisations
(à gauche : $\mathcal{P}(1)$, à droite : $\mathcal{P}(5)$).
```

La loi de Poisson est généralement utilisée lors d'un comptage d'objets
(comme des photons ou des électrons).
C'est donc une loi particulièrement utile dans des signaux électroniques.
Toutefois, lorsque le nombre d'objets comptés est grand,
le bruit peut être modélisé par une distribution de probabilité gaussienne,
qui est plus simple à manipuler mathématiquement.


### Caractéristiques statistiques d'une variable aléatoire

* L'_espérance_ d'une variable aléatoire $x$, notée $\mathbb{E}[x]$, correspond à la valeur moyenne des réalisations :
  
  $$
  \mathbb{E}[x] = \int x p(x) dx.
  $$
  
  ```{margin}
  L'accent $\hat{\cdot}$ signifie « estimation ».
  ```
  
  L'espérance est le
  [moment d'ordre 1](https://fr.wikipedia.org/wiki/Moment_(probabilit%C3%A9s)#Moment_ordinaire)
  de la variable aléatoire $x$.
  
  Lorsqu'on dispose de $N$ réalisations $x[n]$ l'espérance peut être estimée ainsi :
  
  $$
  \hat{\mathbb{E}}[x] = \frac{1}{N} \sum_{n=0}^{N-1} x[n].
  $$
  
* ```{margin}
  Pourquoi la dispersion est considérée comme le carré de la différence
  et pas simplement la différence ?
  La raison est que le carré permet d'avoir des erreurs qui ne peuvent pas être négatives
  tout en facilitant la manipulation lors des calculs, contrairement à la valeur absolue.
  ```
  La _variance_ d'une variable aléatoire $x$, notée $\mathbb{V}[x]$,
  correspond à l'espérance de la dispersion des réalisations $x$
  autour de la moyenne $\mathbb{E}[x]$.
  Cette dispersion est définie comme le carré de la différence, ainsi :
  
  $$
  \mathbb{V}[x] = \mathbb{E} \Big[ \big(x-\mathbb{E}(x)\big)^2 \Big].
  $$

  On peut montrer que la variance s'écrit également :
  
  $$
  \mathbb{V}[x] = \mathbb{E}(x^2) - \mathbb{E}(x)^2.
  $$

  ```{dropdown} Démonstration
  
  En appliquant la définition de l'espérance puis en développant l'expression :
  
  $$
  \mathbb{V}[x] &= \mathbb{E} \Big[ \big(x-\mathbb{E}(x)\big)^2 \Big] \\
                &= \int \big(x-\mathbb{E}(x)\big)^2 p(x) dx \\
                &= \int \big(x^2 - 2x\mathbb{E}(x) + \mathbb{E}(x)^2\big) p(x) dx.
  $$

  L'intégrale étant linéaire,
  et comme en plus $\mathbb{E}(x)$ est une constante par rapport à $x$, alors :

  $$
  \mathbb{V}[x] = \underbrace{\int x^2 p(x) dx}_{\mathbb{E}(x^2)}
                 - 2\mathbb{E}(x) \underbrace{\int x p(x) dx}_{\mathbb{E}(x)}
                 + \mathbb{E}(x)^2 \underbrace{\int p(x) dx}_{1}
  $$

  En simplifiant l'expression, on trouve bien :

  $$
  \mathbb{V}[x] = \mathbb{E}(x^2) - \mathbb{E}(x)^2.
  $$
  ```

  La variance est le
  [moment centré d'ordre 2](https://fr.wikipedia.org/wiki/Moment_(probabilit%C3%A9s)#Moment_centr%C3%A9)
  de la variable aléatoire $x$.

  ```{margin}
  Il existe plusieurs estimateurs de la variance,
  celui-ci est le plus courant :
  c'est l'estimateur non biaisé de la variance.
  ```

  Lorsqu'on dispose de $N$ réalisations $x[n]$, la variance peut être estimée ainsi :
  
  $$
  \hat{\mathbb{V}}[x] = \frac{1}{N-1} \sum_{n=0}^{N-1} \big(x[n]-\hat{\mathbb{E}}[x]\big)^2.
  $$
  
* L'_écart type_ d'une variable aléatoire $x$ est la moyenne des écarts des réalisations à la moyenne.
  C'est la racine carrée de la variance :
  
  $$
  \sqrt{\mathbb{V}(x)}.
  $$

<a class="exercise btn btn-light" href="td.html#exercice-1" role="button">1</a>
<a class="exercise btn btn-light" href="td.html#exercice-2" role="button">2</a>

## Densité spectrale de puissance

### Autocorrélation statistique

L'autocorrélation statistique adapte l'[autocorrélation temporelle](https://vincmazet.github.io/signal1/xcorr/intercorrelation.html#autocorrelation) au caractère statistique d'un signal aléatoire.
Ainsi pour un signal aléatoire $x$, son autocorrélation statistique s'écrit :

$$
R_x[m] = \mathbb{E}\Big[ x[n+m] x[n] \Big].
$$

### Densité spectrale de puissance

La densité spectrale de puissance (_power spectral density_) est la transformée de Fourier à temps discret de l'autocorrélation statistique :

$$
S_x(f) = \mathcal{F} \left[ R_x \right](f) = \sum_{m=-\infty}^{+\infty} R_x[m] e^{-j 2 \pi m f}.
$$

La densité spectrale de puissance est définie pour $f\in\left[-\frac{1}{2},+\frac{1}{2}\right]$.
Elle indique comment la puissance du signal aléatoire se localise en fonction des fréquences.

<!-- Si la vraie définition est que la dsp est la puissance répartie sur les fréquences, comment faire le lien avec la transformée de Fourier de l'autocorrélation ? -->

<!-- (EXEMPLES) -->

La densité spectrale de puissance a les propriétés suivantes.

* La densité spectrale de puissance est une fonction réelle : $S_x(f) \in \mathbb{R}$.

* La densité spectrale de puissance est une fonction positive : $S_x(f) \geq 0$.

* L'intégrale de la densité spectrale de puissance est égale à l'autocorrélation en 0 mais aussi à la puissance du signal :
  
  $$
  \int_{-1/2}^{+1/2} S_x(f) df = R_x(0) = P_x
  $$

* La densité spectrale de puissance ne caractérise pas entièrement un signal aléatoire : deux signaux aléatoires peuvent avoir la même densité spectrale de puissance mais être différents !


<!-- 
Démo

$$
\int_{-1/2}^{+1/2} S_x(f) df
= \int_{-1/2}^{+1/2}  e^{+j2 \pi 0 f} df
= \mathcal{F}^{-1} \left[ S_x \right](0)
= R_x(0)
$$

Utiliser Parseval pour faire le lien avec P_x,
en notant qu'en intégrant la densité (de puissance), on obtient le tout (la puissance).

-->

## Le bruit dans les signaux

On appelle _bruit_ les modifications non souhaitées et inconnues
que subit un signal lors de son acquisition, de son enregistrement, de son traitement ou de sa transmission.

Ce qu'on appelle bruit dépend très fortement du contexte [[Jutten 2018]](C:refs).
Par exemple, pour l'opérateur sonar d'un sous-marin militaire,
le signal utile est émis par les navires alors que les bancs de poissons
émettent des signaux qui sont des perturbations pour le signal utile, donc du bruit.
À l'inverse, pour l'opérateur sonar d'un navire de pêche,
le signal utile est celui émis par les bancs de poissons
et les autres signaux sont des perturbations et constituent du bruit.

Le bruit est donc un signal qui va perturber le signal original.
Comme les modifications apportées par le bruit sont inconnues,
alors ce dernier est modélisé par un signal aléatoire dont les amplitudes sont des variables aléatoires.
La densité de probabilité du bruit peut être gaussienne, uniforme, de Poisson, etc.


### Le bruit blanc

Un bruit $b[n]$ _blanc_ est un signal aléatoire d'espérance nulle et de densité spectrale de puissance constante.
Cela implique que la puissance du bruit est répartie de manière uniforme sur les fréquences, justifiant l'appelation de « blanc » par analogie avec la lumière.

En notant $\sigma_b^2$ la valeur constante de la densité spectrale de puissance,
l'autocorrélation du bruit blanc $b$ est nulle partout sauf en $0$ :

$$
S_b(f) = \sigma_b^2
\quad\Leftrightarrow\quad
R_b[m] = \sigma_b^2 \delta[m].
$$

Ce résultat implique que les échantillons d'un bruit blanc sont décorrélés.

La valeur de l'autocorrélation en $0$ étant égale à la puissance du signal,
le bruit blanc a pour puissance $\sigma_b^2$.

Cette valeur $\sigma_b^2$ est la variance du signal $b$ puisque $R_b[0] = \sigma_b^2 \delta[0]$
et que, sachant que $\mathbb{E}[b] = 0$ on a :

$$
R_b[0] = \mathbb{E}\Big[ b[n+0] b[n] \Big] = \mathbb{E}\Big[ b^2 \Big] = \mathbb{V}\Big[ b \Big].
$$


### Le bruit blanc i.i.d.

un bruit blanc $b[n]$ est _indépendant et identiquement distribué_ (i.i.d.) si :
- ses échantillons sont sans relation les uns avec les autres (ils sont indépendants),
- ses échantillons ont tous la même densité de probabilité (ils sont identiquement distribués).


### Le bruit blanc gaussien additif

Le bruit blanc gaussien additif (AWGN : _additive white Gaussian noise_)
est un modèle très répandu et assez simple de bruit qui possède les caractéristiques suivantes :
- le bruit est blanc,
- ses réalisations sont i.i.d.,
- la densité de probabilité de ses réalisations est une loi normale,
- le bruit s'ajoute au signal d'intérêt.

Ainsi, un signal $x$ bruité par un bruit AWGN se modélise :

$$
\forall n, \quad
y[n] = x[n] + b[n]\quad
\text{où}\; b[n] \sim \mathcal{N}(0,\sigma).
$$

<div class="exemple admonition">
    <!-- Obligé de passer par ce hack car exemple sur plusieurs cellules -->
    <p class="admonition-title">Exemple</p>

Dans cet exemple, une sinusoïde $x$ de 440 Hz (correspondant à la note _la_)
est bruité par un bruit blanc gaussien additif $b$.
Les signaux sont représentés dans la {numref}`F:bruit:exemple-bruit-blanc`
et peuvent être écoutés ci-après.
    
```{figure} exemple-bruit-blanc.svg
---
width: 100%
name: F:bruit:exemple-bruit-blanc
---
Signal $x$, bruit blanc $b$, et la somme $x+b$.
```

In [12]:
from scipy.io.wavfile import read
from IPython.display import Audio

Signal $x$ :

In [9]:
fe, x = read("x.wav")
Audio(x, rate=fe)

Signal $b$ :

In [10]:
fe, b = read("b.wav")
Audio(b, rate=fe)

Signal $y$ :

In [11]:
fe, y = read("y.wav")
Audio(y, rate=fe)

<!-- :::: -->
</div>

### Rapport signal à bruit

Le rapport signal-à-bruit (RSB, ou en anglais SNR : _signal-to-noise ratio_) est une mesure de l'intensité du bruit dans un signal.
Il correspond au rapport entre la puissance du signal $x$ et la puissance du bruit $b$
et est généralement exprimé en décibels :

$$
\text{RSB} = 10 \log_{10} \left(\frac{P_x}{P_b}\right)
$$

où les puissances s'expriment :

$$
P_x = \frac{1}{N} \sum_{n=0}^{N-1} |x[n]|^2
\quad\text{et}\quad
P_b = \frac{1}{N} \sum_{n=0}^{N-1} |b[n]|^2.
$$

Notez que dans le cas d'un bruit blanc, la puissance du bruit $P_b$ est égale à la variance du bruit.

```{figure} rsb.svg
---
width: 100%
name: F:ssp:rsb
---
Exemples d'une même sinusoïde bruitée à différents RSB.
```

Le RSB n'est bien évidemment pas utilisé dans un traitement réel,
puisque le bruit $b$ n'est pas connu.
Le RSB est utilisé pendant la phase de développement d'une méthode de traitement du signal.
En effet, les tests effectués durant cette phase de développement
nécessitent d'avoir à disposition le signal non bruité et le bruit.
Il est alors possible de calculer le RSB,
ce qui permet de comparer les performances du traitement réalisé avec d'autres méthodes.

<a class="exercise btn btn-light" href="td.html#exercice-3" role="button">3</a>